# Learning and Decision Making

## Laboratory 6: Reinforcement learning

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. The windy gridworld domain

Consider the larger version of the windy gridworld domain depicted in the figure below.

<img src="windy.png" width="400px">

In it, a boat must navigate a 7 &times; 10 gridworld, to reach the goal cell, marked with _G_. There is a crosswind upward through the middle of the grid, in the direction indicated by the gray arrows. The boat has available the standard four actions -- _Up_, _Down_, _Left_ and _Right_. In the region affected by the wind, however, the resulting next state is shifted upward as a consequence of the crosswind, the strength of which varies from column to column. The strength of the wind is given below each column, and corresponds to the number of cells that the movement is shifted upward. For example, if the boat is one cell to the right of the goal, then the action _Left_ takes you to the cell just above the goal.

The agent pays a cost of 1 in every step before reaching the goal. The problem can be described as an MDP $(\mathcal{X},\mathcal{A},\mathbf{P},c,\gamma)$ as follows.

In [0]:
%matplotlib notebook
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

np.set_printoptions(threshold=10)

# Problem specific parameters
WIND = (0, 0, 0, 1, 1, 1, 2, 2, 1, 0)
nrows = 7
ncols = 10
init = [3, 0]
goal = [3, 7]

# States
X = [[x, y] for x in range(nrows) for y in range(ncols)]
nX = len(X)

# Actions
A = ['U', 'D', 'L', 'R']
nA = len(A)

# Transition probabilities
P = dict()
P['U'] = np.zeros((nX, nX))
P['D'] = np.zeros((nX, nX))
P['L'] = np.zeros((nX, nX))
P['R'] = np.zeros((nX, nX))

for i in range(len(X)):
    x = X[i]
    y = dict()
    
    y['U'] = [x[0] - WIND[x[1]] - 1, x[1]]
    y['D'] = [x[0] - WIND[x[1]] + 1, x[1]]
    y['L'] = [x[0] - WIND[x[1]], x[1] - 1]
    y['R'] = [x[0] - WIND[x[1]], x[1] + 1]
    
    for k in y:
        y[k][0] = max(min(y[k][0], nrows - 1), 0)
        y[k][1] = max(min(y[k][1], ncols - 1), 0)
        j = X.index(y[k])
        P[k][i, j] = 1

c = np.ones((nX, nA))
c[X.index(goal), :] = 0

gamma = 0.99

# -- Pretty print

print('\n- MDP problem specification: -\n')

print('States:')
print(np.array(X))

print('\nActions:')
print(A)

print('\nTransition probabilities:')
for a in A:
    print('Action', a)
    print(P[a])
    
print('\ncost:')
print(c)
#for i, c1 in enumerate(c):
#    print(i, "->", c1)
    
print('\nStart state:', init)
print('\nGoal state:', goal)

---

#### Activity 1.        

Compute the optimal _Q_-function for the MDP defined above using value iteration. As your stopping condition, use an error between iterations smaller than `1e-8`.

---

In [0]:
e = pow(10, -8)

err = 1
J = np.zeros((nX, 1))

cU = c[:, 0].reshape(nX, 1)
cD = c[:, 1].reshape(nX, 1)
cL = c[:, 2].reshape(nX, 1)
cR = c[:, 3].reshape(nX, 1)

qU = np.zeros((nX, 1))
qD = np.zeros((nX, 1))
qL = np.zeros((nX, 1))
qR = np.zeros((nX, 1))

while (err > 1e-8):
    qU = cU + gamma * P['U'].dot(J)
    qD = cD + gamma * P['D'].dot(J)
    qL = cL + gamma * P['L'].dot(J)
    qR = cR + gamma * P['R'].dot(J)
    newJ = np.min((qU, qD, qL, qR), axis = 0)
    err = np.linalg.norm(newJ - J)
    J = newJ
    i = i + 1

optimal_Q = np.concatenate((qU,qD,qL,qR), axis=1)

print("Q-function: \n", optimal_Q)

---

#### Activity 2.        

Write down a Python function that, given a Q-function $Q$ and a state $x$, selects a random action using the $\epsilon$-greedy policy obtained from $Q$ for state $x$. Your function should receive an optional parameter, corresponding to $\epsilon$, with default value of 0.1. 

**Note:** In the case of two actions with the same value, your $\epsilon$-greedy policy should randomize between the two.

---

In [0]:
from random import uniform, randint

def greedy(q_function, x, epsilon = 0.1):
    q_line = q_function[X.index(x)]
    rand = uniform(0, 1)
    max_action_index = np.argmax(q_line)
    random_action = randint(0, len(q_line) - 1)
    action = A[max_action_index] if rand <= epsilon else A[random_action]
    return action

### 2. Model-based learning

You will now run the model-based learning algorithm discussed in class, and evaluate its learning performance.

---

#### Activity 3.        

Run the model-based reinforcement learning algorithm discussed in class to compute $Q^*$ for 100,000 iterations. Initialize each transition probability matrix as the identity and the cost function as all-zeros. Use an $\epsilon$-greedy policy with $\epsilon=0.1$ (use the function from Activity 2). Note that, at each step,

* You will need to select an action according to the $\epsilon$-greedy policy;
* The state and action, you will then compute the cost and generate the next state; 
* With this transition information (state, action, cost, next-state), you can now perform an update. 
* When updating the components $(x,a)$ of the model, use the step-size

$$\alpha_t=\frac{1}{N_t(x,a)+1},$$

where $N_t(x,a)$ is the number of visits to the pair $(x,a)$ up to time step $t$.

In order to ensure that your algorithm visits every state and action a sufficient number of times, after the boat reaches the goal cell, make one further step, the corresponding update, and then reset the position of the boat to a random state in the environment.

Plot the norm $\|Q^*-Q^{(k)}\|$ every 500 iterations of your method, where $Q^*$ is the optimal _Q_~function computed in Activity 1.

**Note:** The simulation may take a bit. Don't despair.

---

In [0]:
import matplotlib.pyplot as plt

new_P = dict()
new_P['U'] = np.identity(nX)
new_P['D'] = np.identity(nX)
new_P['L'] = np.identity(nX)
new_P['R'] = np.identity(nX)

new_C = np.zeros((nX,nA))

new_Q = np.zeros((nX, nA))

N = np.zeros((nX, nA))

current_X = init
Q_difference_y = []
Q_difference_x = []
random_init = False

for i in range(100000):
    #select action
    current_action = greedy(new_Q, current_X)
    
    #apply action -> new state
    #it's deterministic, but let's do it like this
    pX = P[current_action][X.index(current_X)]
    new_X = X[np.random.choice(np.arange(nX), 1, p = pX)[0]]
    
    #indexes
    i_X = X.index(current_X)
    i_new_X = X.index(new_X)
    i_A = A.index(current_action)
    
    #new step
    N[i_X, i_A] = N[i_X, i_A] + 1
    step = 1 / (N[i_X, i_A] + 1)
    
    #update cost
    new_C[i_X, i_A] = new_C[i_X, i_A] + step*(c[i_X, i_A] - new_C[i_X, i_A])
        
    #update P
    p_sum = np.zeros(nX)
    p_sum[i_new_X] = 1
    new_P[current_action][i_X] = new_P[current_action][i_X] + step*( p_sum - new_P[current_action][i_X])
    new_P[current_action][i_X] = new_P[current_action][i_X] / np.sum(new_P[current_action][i_X])
    
    #update Q
    new_Q[i_X, i_A] = new_C[i_X, i_A] + new_P[current_action][i_X].dot( np.min(new_Q, axis = 1) )
    
    #update state
    if (random_init):
        random_init = False
        new_X = X[np.random.randint(nX)]
    
    elif (current_X == goal):
        random_init = True

    current_X = new_X
    
    #plot ya later
    if (i%500 == 0):
        Q_difference_x.append(i)
        Q_difference_y.append(np.linalg.norm(optimal_Q - new_Q))
        
#print(Q_difference_y)
#print(new_Q)

plt.figure()
plt.plot(Q_difference_x, Q_difference_y)
plt.show()

### 3. Temporal-difference learning

You will now run both Q-learning and SARSA, and compare their learning performance with that of the model-based method just studied.

---

#### Activity 4.        

Repeat Activity 3 but using the _Q_-learning algorithm with a learning rate $\alpha=0.3$.

---

---

#### Activity 5.

Repeat Activity 4 but using the SARSA algorithm.

---

---

#### Activity 6.

Discuss the differences observed between the performance of the three methods.

---

_Add your discussion here._